# Домашняя работа 3. DBScan кластеризация.

Вариант 2. Economics of Cities

Лазарев Эдуард Артемович
N33471

In [1]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [12, 12]

In [2]:
# Загружаем данные
data = pd.read_csv("Economics of Cities/Econom_Cities_data.csv", decimal=",", delimiter=";", index_col='City')
len(data)

48

Проверяем правильность данных

In [3]:
data

,Work,Price,Salary
City,,,
Amsterdam,1714,65.6,49.0
Athens,1792,53.8,30.4
Bogota,2152,37.9,11.5
Bombay,2052,30.3,5.3
Brussels,1708,73.8,50.5
Buenos_Aires,1971,56.1,12.5
Cairo,-9999,37.1,-9999.0
Caracas,2041,61.0,10.9
Chicago,1924,73.9,61.9


In [4]:
data.dtypes

Work        int64
Price     float64
Salary    float64
dtype: object

Видим что есть два города для которых неизвестно значение работы и заработной платы. Удалим их как выбросы.

In [5]:
X = data.where(data.values != -9999).dropna()
X

,Work,Price,Salary
City,,,
Amsterdam,1714.0,65.6,49.0
Athens,1792.0,53.8,30.4
Bogota,2152.0,37.9,11.5
Bombay,2052.0,30.3,5.3
Brussels,1708.0,73.8,50.5
Buenos_Aires,1971.0,56.1,12.5
Caracas,2041.0,61.0,10.9
Chicago,1924.0,73.9,61.9
Copenhagen,1717.0,91.3,62.9


Видим что в первом столбце у нас даны часы, а во втором и третьем стобце проценты, причём для Цюриха значение взято за 100%. Стандартизуем первый столбец, так чтобы для Цюриха было тоже 100%.

In [6]:
X["Work"] = X["Work"] / X["Work"]["Zurich"] * 100
X

,Work,Price,Salary
City,,,
Amsterdam,91.755889,65.6,49.0
Athens,95.931478,53.8,30.4
Bogota,115.203426,37.9,11.5
Bombay,109.850107,30.3,5.3
Brussels,91.434690,73.8,50.5
Buenos_Aires,105.513919,56.1,12.5
Caracas,109.261242,61.0,10.9
Chicago,102.997859,73.9,61.9
Copenhagen,91.916488,91.3,62.9


Проведём DBScan кластеризацию.

In [7]:
from sklearn.cluster import DBSCAN

В качестве начальных значений выберем следующие:
$$\varepsilon = 15 \\ m = D + 1 = 4$$, где $D$ - кол-во столбцов в исходных данных. 

In [8]:
dbscan_1 = DBSCAN(eps=15, min_samples=4)
dbscan_1.fit(X)
unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  8]
 [ 0 19]
 [ 1 19]]


У нас получилось два кластера, и при этом не очень много выбросов. Но разбиение на два кластера слишком малеькое. Посмотрим на другие варианты кластеризации при $1 \leq \varepsilon \leq 50$, $3 \leq m \leq 10$. При этом сразу будем отсекать варианты с разделением меньше чем на три кластера или же если выбросов больше чем элементов в любом из кластеров. Сохраним только уникальные разбиения на кластеры.

In [9]:
previous_results = []
for m in range(3, 10):
    for e in range(1, 50):
        dbscan_iter = DBSCAN(eps=e, min_samples=3)
        dbscan_iter.fit(X)
        unique, counts = np.unique(dbscan_iter.labels_, return_counts=True)
        if len(unique) < 4 or counts[0] >= max(counts[1:]):
            continue
        res = np.asarray((unique, counts)).T

        if any(np.array_equal(res, prev_res) for prev_res in previous_results):
            continue
        
        previous_results.append(np.asarray((unique, counts)).T)
        print(f"{e=}, {m=}:")
        print("\t" + str(res).replace("\n", "\n\t"))
        print()


e=11, m=3:
	[[-1 14]
	 [ 0 16]
	 [ 1  4]
	 [ 2 12]]

e=12, m=3:
	[[-1  9]
	 [ 0 16]
	 [ 1  4]
	 [ 2 14]
	 [ 3  3]]

e=13, m=3:
	[[-1  6]
	 [ 0 18]
	 [ 1 19]
	 [ 2  3]]

e=14, m=3:
	[[-1  5]
	 [ 0 19]
	 [ 1 19]
	 [ 2  3]]



Видим что получилось четыре варианта разбиения на кластеры, удовлетворяющие нашим поставленным условиям.

При $\varepsilon = 11$ мы получаем много выбросов, поэтому отбросим этот вариант. У нас остаётся на выбор три варианта. Если приглядется то при $\varepsilon = 13$ и $\varepsilon = 14$ варианты отличается только одним городом. Вариант, когда $\varepsilon$ принимает значение 12, имеет почти такое же разделение как и два предыдущих, но он выделяет дополнительно ещё один кластер. Выберем его, как результирующее разбиение и посмотрим какие кластеры у нас образовались.

In [10]:
dbscan_2 = DBSCAN(eps=12, min_samples=3)
dbscan_2.fit(X)
unique, counts = np.unique(dbscan_2.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  9]
 [ 0 16]
 [ 1  4]
 [ 2 14]
 [ 3  3]]


In [11]:
X["dbscan_2"] = dbscan_2.labels_
X

,Work,Price,Salary,dbscan_2
City,,,,
Amsterdam,91.755889,65.6,49.0,0
Athens,95.931478,53.8,30.4,1
Bogota,115.203426,37.9,11.5,2
Bombay,109.850107,30.3,5.3,2
Brussels,91.434690,73.8,50.5,0
Buenos_Aires,105.513919,56.1,12.5,2
Caracas,109.261242,61.0,10.9,2
Chicago,102.997859,73.9,61.9,0
Copenhagen,91.916488,91.3,62.9,-1


In [12]:
X[["Work", "Price", "Salary"]].describe()

,Work,Price,Salary
count,46.000000,46.000000,46.000000
mean,100.637743,70.100000,39.545652
std,9.333113,21.389177,24.757703
min,84.743041,30.300000,2.700000
25%,93.428801,51.775000,14.375000
50%,98.982869,70.950000,43.650000
75%,105.794968,81.900000,59.700000
max,127.141328,115.500000,100.000000


In [13]:
X.groupby("dbscan_2").std()

,Work,Price,Salary
dbscan_2,,,
-1,11.583158,18.297609,28.254872
0,6.153538,6.774680,7.203769
1,3.534002,4.402556,3.699099
2,7.716428,10.336431,6.174620
3,8.195825,0.984886,2.193171


In [14]:
for cluster in sorted(set(X["dbscan_2"].tolist())):
    print(f"In cluster {cluster}:\n", X[X["dbscan_2"] == cluster].index.values)

In cluster -1:
 ['Copenhagen' 'Geneva' 'Hong_Kong' 'Houston' 'Lisbon' 'Luxembourg'
 'Stockholm' 'Taipei' 'Zurich']
In cluster 0:
 ['Amsterdam' 'Brussels' 'Chicago' 'Dublin' 'Dusseldorf' 'Frankfurt'
 'London' 'Los_Angeles' 'Madrid' 'Milan' 'Montreal' 'New_York' 'Paris'
 'Sydney' 'Toronto' 'Vienna']
In cluster 1:
 ['Athens' 'Johannesburg' 'Nicosia' 'Seoul']
In cluster 2:
 ['Bogota' 'Bombay' 'Buenos_Aires' 'Caracas' 'Kuala_Lumpur' 'Lagos'
 'Manila' 'Mexico_City' 'Nairobi' 'Panama' 'Rio_de_Janeiro' 'San_Paulo'
 'Singpore' 'Tel_Aviv']
In cluster 3:
 ['Helsinki' 'Oslo' 'Tokyo']


In [15]:
X.groupby("dbscan_2").mean()

,Work,Price,Salary
dbscan_2,,,
-1,102.771830,82.866667,54.544444
0,95.640391,77.418750,54.231250
1,99.089936,52.775000,28.850000
2,107.368461,48.921429,10.485714
3,91.541756,114.700000,66.100000


In [16]:
X.groupby("dbscan_2").size()

dbscan_2
-1     9
 0    16
 1     4
 2    14
 3     3
dtype: int64

## Вывод

Получилась неплохая кластеризация, выделившая нам четыре кластера и девять выбросов. Если посмотреть на значения стандартных отклонений в полученных кластерах, можно заметить что у нас не нашлось ленточных кластеров, и даже если разброс по зарплатам и ценам у выбросов был бы чучуть меньше, то можно было бы считать их за отдельных кластер. 

Результаты этой работы не сильно отличаются от результатов иерархического кластерноного анализа и от кластеризации методом k-means, что говорит о том, что кластеризация скорее всего проведена верно. 

Смотря на полученную статистику по кластерам можно заметить что все города работают почти одинаково и значение столбца с часами работы скорее всего не сильно влияло на кластеризацию.

Далее следует интерпретация самих кластеров.

Начнём со второго кластера. в нём у нас находятся города с самым низким значением заработной платы (в среднем меньше в пять раз по сравнению со средним значение по всей выборке) и цен на продукты. В данный кластер вошли города южных стран, что логично сочетается с полученными результатами.

В первом кластере у нас города в которых экономическая обстановка немного лучше третьего кластера (цены на продукты сравнимы, а зарплата почти в три раза выше), но всё ещё низкая.

В нулевом кластере у нас города со средним значением по всей выборке. Так же можно заметить что таких городов большинство.

В третьем кластере у нас находятся города в которых самая большая цена на продучты и при этом низкие зарплаты (в два раза меньше средней цены на продукты).
